In [2]:
# Optional cell to reduce the amount of logs

import logging

logger = logging.getLogger("unstructured.ingest")
# logger.root.removeHandler(logger.root.handlers[0])

In [5]:
import os

from unstructured.ingest.connector.local import SimpleLocalConfig
from unstructured.ingest.interfaces import PartitionConfig, ProcessorConfig, ReadConfig
from unstructured.ingest.runner import LocalRunner

output_path = "./local-ingest-output"

runner = LocalRunner(
    processor_config=ProcessorConfig(
        # logs verbosity
        verbose=True,
        # the local directory to store outputs
        output_dir=output_path,
        num_processes=2,
    ),
    read_config=ReadConfig(),
    partition_config=PartitionConfig(
        partition_by_api=True,
        api_key="i70Bdasv37rbUVRs4HCdrEq5b1BNl0",
    ),
    connector_config=SimpleLocalConfig(
        input_path="./documents",
        # whether to get the documents recursively from given directory
        recursive=False,
    ),
)
runner.run()

2024-08-01 17:12:58,203 MainProcess INFO     running pipeline: DocFactory -> Reader -> Partitioner -> Copier with config: {"reprocess": false, "verbose": true, "work_dir": "/Users/pangmengting/.cache/unstructured/ingest/pipeline", "output_dir": "./local-ingest-output", "num_processes": 2, "raise_on_error": false}
2024-08-01 17:12:58,279 MainProcess INFO     Running doc factory to generate ingest docs. Source connector: {"processor_config": {"reprocess": false, "verbose": true, "work_dir": "/Users/pangmengting/.cache/unstructured/ingest/pipeline", "output_dir": "./local-ingest-output", "num_processes": 2, "raise_on_error": false}, "read_config": {"download_dir": "", "re_download": false, "preserve_downloads": false, "download_only": false, "max_docs": null}, "connector_config": {"input_path": "./documents", "recursive": false, "file_glob": null}}
2024-08-01 17:12:58,280 MainProcess INFO     no docs found to process


In [ ]:
from unstructured_client import UnstructuredClient

unstructured_api_key = os.environ.get("UNSTRUCTURED_API_KEY")

client = UnstructuredClient(
    api_key_auth=unstructured_api_key,
    # if using paid API, provide your unique API URL:
    # server_url="YOUR_API_URL",
)

In [ ]:
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError
from unstructured.staging.base import dict_to_elements

path_to_pdf = "PATH_TO_YOUR_PDF_FILE"

with open(path_to_pdf, "rb") as f:
    files = shared.Files(
        content=f.read(),
        file_name=path_to_pdf,
    )
    req = shared.PartitionParameters(
        files=files,
        chunking_strategy="by_title",
        max_characters=512,
    )
    try:
        resp = client.general.partition(req)
    except SDKError as e:
        print(e)

elements = dict_to_elements(resp.elements)

In [ ]:
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

documents = []
for element in elements:
    metadata = element.metadata.to_dict()
    documents.append(Document(page_content=element.text, metadata=metadata))

db = FAISS.from_documents(documents, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})